In [4]:
cd '/projects/home-credit-default-risk/'

/projects/home-credit-default-risk


In [5]:
from src import proj_utils, utils, classes

/root/anaconda3/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)
/root/anaconda3/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)


In [6]:
from importlib import reload
from pprint import pprint

In [7]:
from lightgbm.sklearn import LGBMClassifier

In [8]:
data_path = '/vol2/competitions/home-credit-default-risk/'

In [9]:
# Load data
df_train = proj_utils.load_data(data_path= data_path,
                                train = True,
                                supp_dict = {'previous_application.csv.zip' : 'max',
                                             'credit_card_balance.csv.zip' : 'mean',
                                             'installments_payments.csv.zip' : 'min',
                                             'POS_CASH_balance.csv.zip' : 'mean',
                                             'bureau.csv.zip' : 'max'
                                            })

df_test = proj_utils.load_data(data_path = data_path,
                               train = False,
                               supp_dict = {'previous_application.csv.zip' : 'max',
                                            'credit_card_balance.csv.zip' : 'mean',
                                            'installments_payments.csv.zip' : 'min',
                                            'POS_CASH_balance.csv.zip' : 'mean',
                                            'bureau.csv.zip' : 'max'
                                           })

Loading training applications
Loading previous_application.csv.zip
Loading credit_card_balance.csv.zip
Loading installments_payments.csv.zip
Loading POS_CASH_balance.csv.zip
Loading bureau.csv.zip
Loading test applications
Loading previous_application.csv.zip
Loading credit_card_balance.csv.zip
Loading installments_payments.csv.zip
Loading POS_CASH_balance.csv.zip
Loading bureau.csv.zip


In [10]:
data = classes.Dataset(df_train, df_test, 'TARGET')

...creating training matrix
...creating test matrix


In [11]:
from sklearn.preprocessing import Imputer
from sklearn.pipeline import Pipeline

In [12]:
from sklearn.linear_model import LogisticRegression

In [13]:
from sklearn.feature_selection import SelectFromModel

In [14]:
preprocessor = Pipeline([('impute', Imputer())])

In [15]:
X = preprocessor.fit_transform(data.X_train)

In [16]:
%time
selector = SelectFromModel(clf, threshold=0.2, )

CPU times: user 2 µs, sys: 1e+03 ns, total: 3 µs
Wall time: 3.81 µs


NameError: name 'clf' is not defined

In [18]:
X.shape

(307511, 837)

In [19]:
from sklearn.model_selection import learning_curve

In [20]:
import numpy as np

In [21]:
%env JOBLIB_TEMP_FOLDER=/vol2/tmp

env: JOBLIB_TEMP_FOLDER=/vol2/tmp


In [ ]:
%time
sizes = list(np.linspace(int(X.shape[0] /10), X.shape[0] * 0.75, 10, dtype=int))
clf = LogisticRegression(penalty = 'l2', solver = 'sag', n_jobs=4, tol= 0.01)
train_sizes, train_scores, valid_scores = learning_curve(clf, 
                                                         X, data.y_train, 
                                                         train_sizes=sizes, cv=5, 
                                                         scoring = 'roc_auc', shuffle = True, random_state = 123, n_jobs = 4, verbose = 2)

In [22]:
init_params = {'boosting_type': 'gbdt',
          'max_depth' : -1,
          'objective': 'binary',
          'learning_rate': 0.0212,
          'reg_alpha': 0.8,
          'reg_lambda': 0.4,
          'subsample': 1,
          'feature_fraction': 0.3,
          'device_type': 'gpu',
          'metric' : 'auc',
          'random_state': 123,
          'n_estimators': 1313, 
          'num_leaves': 40, 
          'max_bin': 255,
          'min_data_in_leaf': 2400,
          'min_data_in_bin': 5}


In [23]:
gbm = LGBMClassifier(**init_params)

In [ ]:
%time
sizes = list(np.linspace(int(X.shape[0] /10), X.shape[0] * 0.75, 10, dtype=int))

train_sizes_2, train_scores_2, valid_scores_2 = learning_curve(clf2, 
                                                         X, data.y_train, 
                                                         train_sizes=sizes, cv=5, 
                                                         scoring = 'roc_auc', shuffle = True, random_state = 123, n_jobs = 4, verbose = 2)

In [24]:
gbm.fit(X,data.y_train)

LGBMClassifier(boosting_type='gbdt', class_weight=None, colsample_bytree=1.0,
        device_type='gpu', feature_fraction=0.3, importance_type='split',
        learning_rate=0.0212, max_bin=255, max_depth=-1, metric='auc',
        min_child_samples=20, min_child_weight=0.001, min_data_in_bin=5,
        min_data_in_leaf=2400, min_split_gain=0.0, n_estimators=1313,
        n_jobs=-1, num_leaves=40, objective='binary', random_state=123,
        reg_alpha=0.8, reg_lambda=0.4, silent=True, subsample=1,
        subsample_for_bin=200000, subsample_freq=0)

In [25]:
def get_percentile_rank(x):
    x_sum = x.sum()
    y = x/x_sum
    init_order = np.argsort(y)
    
    y_sorted = sorted(y, reverse = True)
    y_cdf = np.cumsum(y_sorted)
    
    return(y_cdf[init_order], y)

In [28]:
cdf, pdf = get_percentile_rank(gbm.feature_importances_)

In [32]:
best_features_idx = cdf >0.99

In [30]:
from sklearn.naive_bayes import GaussianNB

In [ ]:
%time
sizes = list(np.linspace(int(X.shape[0] /10), X.shape[0] * 0.75, 10, dtype=int))
#clf = LogisticRegression(penalty = 'l2', solver = 'sag', n_jobs=4, tol= 0.001)
clf = GaussianNB()
train_sizes, train_scores, valid_scores = learning_curve(clf, 
                                                         X[:, ], data.y_train, 
                                                         train_sizes=sizes, cv=5, 
                                                         scoring = 'roc_auc', shuffle = True, random_state = 123, n_jobs = 4, verbose = 2)

In [33]:
nb = GaussianNB().fit(X[:, best_features_idx], data.y_train)

In [34]:
from sklearn.model_selection import train_test_split

In [44]:
from sklearn.metrics import roc_auc_score

In [35]:
X_train, X_test, y_train, y_test = train_test_split(X[:, best_features_idx], data.y_train, test_size=0.33, random_state=42)

In [69]:
X_train_1, X_holdout, y_train_1, y_holdout = train_test_split(X_train, y_train, test_size=0.33, random_state=42)

In [70]:
nb = GaussianNB().fit(X_train_1, y_train_1)

In [37]:
from sklearn.ensemble import RandomForestClassifier

In [71]:
rf = RandomForestClassifier(n_estimators = 500).fit(X_train_1, y_train_1)

In [39]:
from sklearn.preprocessing import MinMaxScaler

In [101]:
lr = Pipeline([('Transformer', MinMaxScaler()),
               ('model', LogisticRegression(C=0.5, solver = 'sag', tol=0.0001))])

In [102]:
lr.fit(X_train_1, y_train_1)

/root/anaconda3/lib/python3.6/site-packages/sklearn/linear_model/sag.py:326: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  "the coef_ did not converge", ConvergenceWarning)


Pipeline(memory=None,
     steps=[('Transformer', MinMaxScaler(copy=True, feature_range=(0, 1))), ('model', LogisticRegression(C=0.5, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='ovr', n_jobs=1,
          penalty='l2', random_state=None, solver='sag', tol=0.0001,
          verbose=0, warm_start=False))])

In [103]:
from mlxtend.classifier import StackingClassifier

In [105]:
from sklearn.model_selection import GridSearchCV

In [104]:
sc = StackingClassifier([nb, rf, lr, gbm], meta_classifier = LogisticRegression(C = 0.001), use_probas= True)

In [106]:
params = {'meta-logisticregression__C': [0.1, 10.0]}

grid = GridSearchCV(estimator=sc, 
                    param_grid=params, 
                    cv=5,
                    refit=True, scoring = 'roc_auc')

In [107]:
grid.fit(X_train_1, y_train_1)

/root/anaconda3/lib/python3.6/site-packages/sklearn/linear_model/sag.py:326: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  "the coef_ did not converge", ConvergenceWarning)
/root/anaconda3/lib/python3.6/site-packages/sklearn/linear_model/sag.py:326: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  "the coef_ did not converge", ConvergenceWarning)
/root/anaconda3/lib/python3.6/site-packages/sklearn/linear_model/sag.py:326: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  "the coef_ did not converge", ConvergenceWarning)
/root/anaconda3/lib/python3.6/site-packages/sklearn/linear_model/sag.py:326: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  "the coef_ did not converge", ConvergenceWarning)
/root/anaconda3/lib/python3.6/site-packages/sklearn/linear_model/sag.py:326: ConvergenceWarning: The max_iter was reached which means the coef_ did 

GridSearchCV(cv=5, error_score='raise',
       estimator=StackingClassifier(average_probas=False,
          classifiers=[GaussianNB(priors=None), RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_spl...tures=False, use_clones=True,
          use_features_in_secondary=False, use_probas=True, verbose=0),
       fit_params=None, iid=True, n_jobs=1,
       param_grid={'meta-logisticregression__C': [0.1, 10.0]},
       pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring='roc_auc', verbose=0)

In [109]:
grid.best_params_

{'meta-logisticregression__C': 0.1}

In [99]:
params = {'meta-logisticregression__C': [0.1, 10.0]}

grid = GridSearchCV(estimator=sc, 
                    param_grid=params, 
                    cv=5,
                    refit=True, scoring = 'roc_auc_score')

In [112]:
roc_auc_score(y_holdout, y_sc)

0.7277285456303217

In [75]:
gbm2.fit(X_train_1, y_train_1)

LGBMClassifier(boosting_type='gbdt', class_weight=None, colsample_bytree=1.0,
        device_type='gpu', feature_fraction=0.3, importance_type='split',
        learning_rate=0.0212, max_bin=255, max_depth=-1, metric='auc',
        min_child_samples=20, min_child_weight=0.001, min_data_in_bin=5,
        min_data_in_leaf=2400, min_split_gain=0.0, n_estimators=1313,
        n_jobs=-1, num_leaves=40, objective='binary', random_state=123,
        reg_alpha=0.8, reg_lambda=0.4, silent=True, subsample=1,
        subsample_for_bin=200000, subsample_freq=0)

In [76]:
stacker_params = {'boosting_type': 'gbdt',
          'max_depth' : -1,
          'objective': 'binary',
          'learning_rate': 0.01,
          'reg_alpha': 0.1,
          'reg_lambda': 0.1,
          'subsample': 1,
          'feature_fraction': 0.85,
          'device_type': 'gpu',
          'metric' : 'auc',
          'random_state': 123,
          'n_estimators': 500, 
          'num_leaves': 40, 
          'max_bin': 255,
          'min_data_in_leaf': 2400,
          'min_data_in_bin': 5}


In [77]:
vals = np.vstack((gbm2.predict_proba(X_holdout)[:,1],
                  rf.predict_proba(X_holdout)[:,1],
                  lr.predict_proba(X_holdout)[:,1])).T

In [59]:
from sklearn.svm import SVC
from xgboost.sklearn import XGBClassifier
from sklearn.linear_model import LogisticRegressionCV

In [78]:
#stacker = LGBMClassifier(**stacker_params).fit(vals, y_train)
#stacker = LogisticRegressionCV(Cs = 20, scoring='roc_auc').fit(vals, y_train)
stacker = LogisticRegression(C = 0.0001).fit(vals, y_holdout)
#stacker = XGBClassifier(n_jobs = 4).fit(vals, y_train)

In [79]:
vals = np.vstack((gbm2.predict_proba(X_holdout)[:,1],
                  rf.predict_proba(X_holdout)[:,1],
                  lr.predict_proba(X_holdout)[:,1])).T

In [119]:
np.corrcoef(vals)

KeyboardInterrupt: 

In [82]:
y_stacker = stacker.predict_proba(vals)[:,1]

In [84]:
roc_auc_score(y_holdout, y_stacker)

0.2422858247636864

In [335]:
def compare_models(m1, m2, X, y, q = 21):
    m_df = pd.DataFrame({'y': y, 
                         'm1': m1.predict_proba(X)[:,1], 
                         'm2': m2.predict_proba(X)[:,1]})
 
    m_df['m1_bin'] = pd.cut(m_df.m1, np.linspace(0,1,q))
    m_df['m2_bin'] = pd.cut(m_df.m2, np.linspace(0,1,q)) 
    
    m_agg = m_df.groupby(['m1_bin', 'm2_bin'])['y'].mean().reset_index()
    
    m_agg_mat = m_agg.pivot(index = 'm1_bin', columns = 'm2_bin', values = 'y') \
        .sort_index(axis = 1, ascending = True) \
        .sort_index(axis = 0, ascending = False)
    
    sb.heatmap(m_agg_mat)
    
    
    

In [111]:
compare_models(gbm, lr, X_train, y_train)

NameError: name 'compare_models' is not defined

In [224]:
import seaborn as sb

In [60]:
from matplotlib.pyplot import plot

In [ ]:

# Clean and transform data

# Determine initial feature importances
data.ae_train_model(model = LGBMClassifier(n_estimators= 100, n_jobs=4))

In [ ]:
# Auto-discover ratios weighted by feature importance
data.autoengineer_ratios(ae_params={'boosting_type': 'gbdt',
                                    'max_depth': -1,
                                    'objective': 'binary',
                                    'learning_rate': 0.0212,
                                    'reg_alpha': 0.8,
                                    'reg_lambda': 0.4,
                                    'subsample': 1,
                                    'feature_fraction': 0.3,
                                    'metric': 'auc',
                                    'random_state': 123,
                                    'n_estimators': 300, 
                                    'num_leaves': 40,
                                    'max_bin': 255,
                                    'min_data_in_leaf': 2400,
                                    'min_data_in_bin': 5},
                         n_iter = 10)

In [ ]:
data.ae_discovery_ratios

In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import GaussianNB
from sklearn.pipeline import Pipeline

In [ ]:
from sklearn.preprocessing import FunctionTransformer

In [ ]:
x = data.X_train.values

In [ ]:
import numpy as np

In [ ]:
x_ismissing

In [ ]:
FeatureReducer = FunctionTransformer(func = _feature_reducer, validate= False)

In [ ]:
reload(utils)

In [ ]:
reducer = utils.NanReducer()

In [ ]:
a = reducer.fit_transform(data.X_train)

In [ ]:
b = reducer.transform(data.X_test)

In [ ]:
reload(classes)

In [ ]:
from sklearn.preprocessing import Imputer

In [ ]:
M1 = classes.Model(Pipeline([('impute', Imputer()),
                             ('clf', LogisticRegression())]),
          name = 'logistic regression',
          desc = 'logistic regression base')

M2 = classes.Model(LGBMClassifier(),
          name = "lgbm",
          desc = "lgbm")

M3 = classes.Model(Pipeline([('impute', Imputer()),
                             ('clf', GaussianNB())]),
          name = 'gaussian nb',
          desc = 'gnb')

models = [M1, M2, M3]

In [ ]:
for m in models:
    print(m.name)
    m.train(data.X_train, data.y_train)

In [ ]:
M1.validate_model(data.X_train, data.y_train)


In [ ]:
M2.validate_model(data.X_train, data.y_train)
M3.validate_model(data.X_train, data.y_train)


In [ ]:
M1_features = M2_features = M3_features = list(data.X_train.columns)

In [ ]:
for m in models:
    print(m.validation_scores.mean())

In [ ]:
reload(classes)

In [ ]:
mc = classes.ModelCollection([(M1, M1_features),
                              (M2, M2_features),
                              (M3, M3_features)])

In [ ]:
mc.validate_model(data.X_train, data.y_train)

In [ ]:
mc.validation_scores_stack.get('test_score').mean()

In [ ]:
from sklearn.feature_selection import SelectFromModel

In [ ]:
mdl = LogisticRegression()

In [ ]:
a = SelectFromModel(M1.clf, threshold = 0.25, prefit = True)

In [ ]:
a.estimator

In [ ]:
piper = Pipeline([('impute', Imputer()),
                  ('lr', LogisticRegression(penalty = 'l1'))])

In [ ]:
a = SelectFromModel(piper, threshold = 0.25)

In [ ]:
from sklearn.feature_selection import chi2

In [ ]:
preprocesser = Pipeline([('imputer', Imputer()),
                         ('')])

In [ ]:
data.X_train.describe()

In [ ]:
a = chi2(Imputer().fit_transform(data.X_train), data.y_train)

In [ ]:
a.transform(data.X_train)

In [ ]:
from mlxtend.classifier import StackingClassifier

In [ ]:
a = StackingClassifier([M1, M2], LogisticRegression())

In [ ]:
from sklearn.model_selection import cross_validate

In [ ]:
b = cross_validate(a, )

In [ ]:
#############################################################
models = {}
for m in M:
    # Define a model
    models[m] = {}

    # Tune parameters
    tuner = Tuner(m, data.X_train, data.y_train)
    for k in range(1,5):
        tuner.tune(kappa = k, pbounds = {}, n_iters = 10)
    

# Compare and stack models
mc = ModelCollection(models)